# Crypto vs the Market

## Initial project setup

In [15]:
# Initial imports
import os
import pandas as pd
import hvplot.pandas
import pandas_ta as ta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
%matplotlib inline

In [16]:
# Load the environment variables from the .env file
load_dotenv()

True

In [17]:
# Set the variables for the Alpaca API key and secret key
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [18]:
# Create the Alpaca API REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "V2"
)

## Alpaca API Data Import

#### Market Data

In [19]:
# Set ticker to SPY
tickers = "SPY"

# Set timeframe to 1Day
timeframe = "1Day"

# Set the start and end date
start_date = pd.Timestamp("2018-07-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-07-01", tz="America/New_York").isoformat()

In [20]:
# Use Alpaca get_bars function to get data for SPY
spy_prices_df = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
).df

# Concatenate the ticker dataframe
spy_prices_df = pd.concat([spy_prices_df], axis=1, keys=["SPY"])

# Use .head() to verify
spy_prices_df.head()

SPY                                     \
                             open    high      low   close    volume   
timestamp                                                              
2018-07-02 04:00:00+00:00  269.51  272.04  269.240  271.86  63769156   
2018-07-03 04:00:00+00:00  272.88  272.98  270.420  270.90  42201071   
2018-07-05 04:00:00+00:00  272.16  273.18  270.960  273.11  57441049   
2018-07-06 04:00:00+00:00  273.14  275.84  272.715  275.42  66693896   
2018-07-09 04:00:00+00:00  276.56  277.96  276.500  277.90  51771719   

                                                   
                          trade_count        vwap  
timestamp                                          
2018-07-02 04:00:00+00:00      279127  270.689557  
2018-07-03 04:00:00+00:00      180591  271.716151  
2018-07-05 04:00:00+00:00      260592  272.247814  
2018-07-06 04:00:00+00:00      269826  274.929843  
2018-07-09 04:00:00+00:00      212828  277.260701

#### Crypto Data

In [21]:
# Set ticker to BTCUSD
tickers = "BTCUSD"

# Set timeframe to 1Day
timeframe = "1Day"

# Set the start and end date
start_date = pd.Timestamp("2018-07-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-07-01", tz="America/New_York").isoformat()

In [22]:
# Use the Alpaca get_crypto_bars function to get data for BTCUSD
btc_prices_df = alpaca.get_crypto_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
).df

# Filter out all exchanges other than Coinbase and drop redundant exchange column
btc_prices_df = btc_prices_df[btc_prices_df.exchange == "CBSE"]
btc_prices_df = btc_prices_df.drop(axis=1, columns="exchange")

# Concatenate the ticker dataframe
btc_prices_df = pd.concat([btc_prices_df], axis=1, keys=["BTCUSD"])

# Use .head() to verify
btc_prices_df.head()

BTCUSD                                          \
                              open     high      low    close       volume   
timestamp                                                                    
2018-07-01 05:00:00+00:00  6349.50  6351.00  6270.01  6304.55   643.082606   
2018-07-02 05:00:00+00:00  6304.55  6669.99  6270.01  6611.79  7184.664968   
2018-07-03 05:00:00+00:00  6502.62  6518.25  6411.51  6469.00  1325.274215   
2018-07-04 05:00:00+00:00  6587.46  6700.00  6508.72  6620.99  1345.963438   
2018-07-05 05:00:00+00:00  6621.00  6684.00  6449.50  6532.95  4944.828533   

                                                    
                          trade_count         vwap  
timestamp                                           
2018-07-01 05:00:00+00:00       61723  6325.628835  
2018-07-02 05:00:00+00:00       37641  6556.943445  
2018-07-03 05:00:00+00:00        9573  6463.236479  
2018-07-04 05:00:00+00:00       57950  6606.931106  
2018-07-05 05:00:00+00:00       43786  6565.028002

## Bitcoin vs the S&P500: Long-Term - Jason

## Bitcoin vs the S&P500: Trading - Majid

## Bitcoin vs the S&P500: Monte Carlo Simulations - Whoever gets to it first

## Closing Notes & Analysis